# Using Foursquare to generate a targeted list for B2B sales

## Part 1:  Importing data & exploring ways to slice results from Foursquare

#### Business problem

Business to business sales often involves extensive driving for purposes of scouting neighborhoods and identifying target businesses.  Analyzing neighborhoods and visualizing short lists of businesses can help sales staff to visualize target businesses and determine strategic routes.  This can help to increase the number of target businesses reached in a given day.  In a field where success is directly proportional to the number of businesses reached, this can result in a higher ROI on time invested.  In this example I will pretend that I am working with a startup gluten-free baking company that is looking to develop business channels for selling their gluten-free vegetarian brownie.

#### Data requirements

Data needed would be:

*Zip codes that will be targeted*

The datasource for this will be OpenDataSoft (ODS).  The site hosts a [US Zip Code](https://public.opendatasoft.com/explore/dataset/us-zip-codes/table/) file that can be filtered by cities of interest.  From this file, I will need to collect the following features
- Zipcode:  lists all relevant zip codes
- City:  name of city in which the zip code is located
- Lat:  latitude at center point of zip code
- Long: longitude at center point of zip code

*Foursquare data on business locations*

The assumption here is that top results would reflect popularity to some extent.  More popular businesses would correlate with higher potential sales of gluten-free vegetarian brownies through these business channels.  Features of interest would be:
- Business name
- Location latitude
- Location longitude
- Business type or catogory

#### Data collection

##### Collect zip codes

In this example I will focus on analyzing zip codes in New Orleans.  To start, I'll need to import libraries that will:
- allow me to connect to the OpenDataSoft URL
- convert the data received into a palatable dataframe
- enable dataframe manipulation

In [6]:
import pandas as pd
import json
import requests

# transforming json file into pandas dataframe library
from pandas.io.json import json_normalize

I want to make the API connection flexible, so I will set up variables that enable me to change the city and state.

In [7]:
# set up city and state
city = 'New Orleans'
zipcodes = 66 # the API limit is 10 so for a full list of zip codes I matched the number of search results
state = 'LA'

# set up URL to accept city and state as variables
url = "https://public.opendatasoft.com/api/records/1.0/search/?dataset=us-zip-code-latitude-and-longitude&q={}&rows={}&facet=state&facet=timezone&facet=dst&refine.state={}".format(city, zipcodes, state)

# fetch URL
response = requests.get(url)

# validate that connection was successful
print(response.status_code)

200


The status code signals that the connection was successful.  Now I'm ready to create a pandas dataframe using data collected from the API.  By inspecting the dataset on the website, I've identified that zip code data is stored in the records key.

In [8]:
x = response.json() # returns a json object of the result
zipcodes = x['records'] # assigns records from API call to zipcodes
df = pd.json_normalize(zipcodes) # converts result into a palatable dataframe
df.head() # prints first 5 rows from dataframe

,datasetid,recordid,record_timestamp,fields.city,fields.zip,fields.dst,fields.geopoint,fields.longitude,fields.state,fields.latitude,fields.timezone,geometry.type,geometry.coordinates
0,us-zip-code-latitude-and-longitude,2754d6f177556f7c4bdc2f04c400b102df78d3d8,2018-02-09T16:33:38.603000+00:00,New Orleans,70156,1,"[30.032997, -89.882564]",-89.882564,LA,30.032997,-6,Point,"[-89.882564, 30.032997]"
1,us-zip-code-latitude-and-longitude,bb1c8e44694e2766ad81bd1474f81f5d6234ae66,2018-02-09T16:33:38.603000+00:00,New Orleans,70175,1,"[30.032997, -89.882564]",-89.882564,LA,30.032997,-6,Point,"[-89.882564, 30.032997]"
2,us-zip-code-latitude-and-longitude,1824eb231990370b665fb35eb12ab67171df9bd3,2018-02-09T16:33:38.603000+00:00,New Orleans,70146,1,"[30.032997, -89.882564]",-89.882564,LA,30.032997,-6,Point,"[-89.882564, 30.032997]"
3,us-zip-code-latitude-and-longitude,f83cad30ec0abfc4178b7825d4404a66cef65a44,2018-02-09T16:33:38.603000+00:00,New Orleans,70182,1,"[30.067646, -89.815993]",-89.815993,LA,30.067646,-6,Point,"[-89.815993, 30.067646]"
4,us-zip-code-latitude-and-longitude,c2f3aab82b85ef46d7d26bb61c5714a56b15df25,2018-02-09T16:33:38.603000+00:00,New Orleans,70115,1,"[29.927305, -90.10197]",-90.101970,LA,29.927305,-6,Point,"[-90.10197, 29.927305]"


Next I can filter down the dataframe to show only the features of interest

In [10]:
new_orleans_zipcodes = df[
    [
        'fields.city',
        'fields.zip',
        'fields.longitude',
        'fields.latitude'
    ]
]

new_orleans_zipcodes.head()

,fields.city,fields.zip,fields.longitude,fields.latitude
0,New Orleans,70156,-89.882564,30.032997
1,New Orleans,70175,-89.882564,30.032997
2,New Orleans,70146,-89.882564,30.032997
3,New Orleans,70182,-89.815993,30.067646
4,New Orleans,70115,-90.101970,29.927305


The column names aren't very convenient, so I will change those.

In [11]:
#map old values to new values
col_names = {
    'fields.city': 'city',
    'fields.zip': 'zipcode',
    'fields.longitude': 'long',
    'fields.latitude': 'lat'
}

# pass dictionaries to rename method
zipcode_df = new_orleans_zipcodes.rename(columns=col_names)
zipcode_df.head()

,city,zipcode,long,lat
0,New Orleans,70156,-89.882564,30.032997
1,New Orleans,70175,-89.882564,30.032997
2,New Orleans,70146,-89.882564,30.032997
3,New Orleans,70182,-89.815993,30.067646
4,New Orleans,70115,-90.101970,29.927305


In [12]:
city = zipcode_df['city']

# validate that all zipcodes belong to New Orleans
city.value_counts()

New Orleans    66
Name: city, dtype: int64

There API returned 66 zipcodes, and all of them are located within the city limits of New Orleans.

##### Collect venue data

First I need to define my Foursquare credentials

In [16]:
import fs_secrets

In [17]:
CLIENT_ID = fs_secrets.client_id # your Foursquare ID
CLIENT_SECRET = fs_secrets.client_secret # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 200 # A default Foursquare API limit value

In [18]:
def getNearbyVenues(zipcode, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for zipcode, lat, lng in zip(zipcode, latitudes, longitudes):
            
        # create the API request URL
        fs_url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(fs_url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            zipcode, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['zipcode', 
                  'zip_lat', 
                  'zip_long', 
                  'venue', 
                  'venue_lat', 
                  'venue_long', 
                  'venue_category']
    
    return(nearby_venues)

Create a dataframe named new_orleans_venues and run the function getNearbyVenues using the zipcodes, latitude, and longitude data from the chicagozip_df dataframe created earlier.

In [19]:
neworleans_venues = getNearbyVenues(zipcode=zipcode_df['zipcode'],
                                  latitudes=zipcode_df['lat'],
                                  longitudes=zipcode_df['long']
                                  )

Preview the results by looking at the shape of the dataframe to see how many venues were returned, and the first 5 rows of the dataframe.

In [21]:
print(neworleans_venues.shape)
neworleans_venues.head()

(1300, 7)


,zipcode,zip_lat,zip_long,venue,venue_lat,venue_long,venue_category
0,70156,30.032997,-89.882564,Barkemeyer Law Firm,30.033591,-89.884306,Lawyer
1,70175,30.032997,-89.882564,Barkemeyer Law Firm,30.033591,-89.884306,Lawyer
2,70146,30.032997,-89.882564,Barkemeyer Law Firm,30.033591,-89.884306,Lawyer
3,70182,30.067646,-89.815993,J Martin Seafood,30.067292,-89.818889,Food
4,70182,30.067646,-89.815993,Environmental Analyst,30.067550,-89.812432,Boat or Ferry


I now have a dataframe of 1300 venues within the 66 zip codes in New Orleans.  With a dataframe of Foursquare venues within 1km of each New Orleans zip code enter in place, I'm ready to move forward with narrowing results to create a short-list of venues to target.  Since there is more than one way to do this, I will use part 1 of this data journey, to find the methods with the least computational burden.

In [22]:
# create a new dataframe with venue categories and sort in alphabetical order
venue_categories = neworleans_venues['venue_category'].sort_values()

# drop duplicates
venue_categories.drop_duplicates(inplace=True)

# print a list of categories
venue_categories.tolist()

['Accessories Store',
 'African Restaurant',
 'American Restaurant',
 'Antique Shop',
 'Argentinian Restaurant',
 'Art Gallery',
 'Arts & Crafts Store',
 'Asian Restaurant',
 'BBQ Joint',
 'Bagel Shop',
 'Bakery',
 'Bank',
 'Bar',
 'Basketball Court',
 'Basketball Stadium',
 'Bed & Breakfast',
 'Beer Garden',
 'Big Box Store',
 'Bike Rental / Bike Share',
 'Boat or Ferry',
 'Bookstore',
 'Boutique',
 'Brazilian Restaurant',
 'Breakfast Spot',
 'Brewery',
 'Burger Joint',
 'Bus Stop',
 'Café',
 'Cajun / Creole Restaurant',
 'Campground',
 'Candy Store',
 'Caribbean Restaurant',
 'Cemetery',
 'Cheese Shop',
 'Chinese Restaurant',
 'Chocolate Shop',
 'Clothing Store',
 'Cocktail Bar',
 'Coffee Shop',
 'College Auditorium',
 'College Baseball Diamond',
 'College Gym',
 'College Theater',
 'Comedy Club',
 'Comfort Food Restaurant',
 'Comic Shop',
 'Construction & Landscaping',
 'Convenience Store',
 'Cosmetics Shop',
 'Creperie',
 'Cupcake Shop',
 'Dance Studio',
 'Deli / Bodega',
 'Departm

Reviewing this list, I have identified the following business categories as high potential for developing business channels:
- Coffee Shop
- Café
- Dessert Shop
- Vegetarian / Vegan Restaurant
- Fondue Restaurant
- Gluten-free Restaurant

Next I will explore various methods to filter the dataframe to only include these venue categories, and identify the method with the lowest computational burden.

##### Construct boolean condition

In [23]:
criteria = ((neworleans_venues.venue_category == 'Coffee Shop') |
              (neworleans_venues.venue_category == 'Café') |
              (neworleans_venues.venue_category == 'Dessert Shop') |
              (neworleans_venues.venue_category == 'Vegetarian / Vegan Restaurant') |
              (neworleans_venues.venue_category == 'Fondue Restaurant') |
              (neworleans_venues.venue_category == 'Gluten-free Restaurant'))
             
neworleans_venues[criteria]

,zipcode,zip_lat,zip_long,venue,venue_lat,venue_long,venue_category
10,70115,29.927305,-90.101970,French Truck Cafe,29.929239,-90.104610,Café
45,70115,29.927305,-90.101970,Mojo Coffee House,29.935092,-90.106383,Coffee Shop
46,70115,29.927305,-90.101970,Yogurtland,29.924917,-90.108316,Dessert Shop
49,70115,29.927305,-90.101970,Bearcat Cafe,29.935653,-90.104042,Coffee Shop
58,70115,29.927305,-90.101970,Rook Cafe,29.935027,-90.104727,Café
...,...,...,...,...,...,...,...
1252,70158,29.922905,-90.070922,Common Grounds,29.915917,-90.064533,Coffee Shop
1253,70158,29.922905,-90.070922,PJ's Coffee & Tea,29.929576,-90.076415,Coffee Shop
1264,70158,29.922905,-90.070922,Cherry Coffee,29.929541,-90.077029,Coffee Shop
1289,70158,29.922905,-90.070922,Whatever Coffee,29.929437,-90.076968,Coffee Shop


In [24]:
%timeit criteria = ((neworleans_venues.venue_category == 'Coffee Shop') | (neworleans_venues.venue_category == 'Café') | (neworleans_venues.venue_category == 'Dessert Shop') | (neworleans_venues.venue_category == 'Vegetarian / Vegan Restaurant') | (neworleans_venues.venue_category == 'Fondue Restaurant') | (neworleans_venues.venue_category == 'Gluten-free Restaurant'))

996 µs ± 52.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [25]:
%timeit neworleans_venues[criteria]

123 µs ± 4.38 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


##### Leverage isin selection

In [26]:
categories = ['Coffee Shop','Café','Dessert Shop','Vegetarian / Vegan Restaurant','Fondue Restaurant','Gluten-free Restaurant']
neworleans_venues[neworleans_venues['venue_category'].isin(categories)]

,zipcode,zip_lat,zip_long,venue,venue_lat,venue_long,venue_category
10,70115,29.927305,-90.101970,French Truck Cafe,29.929239,-90.104610,Café
45,70115,29.927305,-90.101970,Mojo Coffee House,29.935092,-90.106383,Coffee Shop
46,70115,29.927305,-90.101970,Yogurtland,29.924917,-90.108316,Dessert Shop
49,70115,29.927305,-90.101970,Bearcat Cafe,29.935653,-90.104042,Coffee Shop
58,70115,29.927305,-90.101970,Rook Cafe,29.935027,-90.104727,Café
...,...,...,...,...,...,...,...
1252,70158,29.922905,-90.070922,Common Grounds,29.915917,-90.064533,Coffee Shop
1253,70158,29.922905,-90.070922,PJ's Coffee & Tea,29.929576,-90.076415,Coffee Shop
1264,70158,29.922905,-90.070922,Cherry Coffee,29.929541,-90.077029,Coffee Shop
1289,70158,29.922905,-90.070922,Whatever Coffee,29.929437,-90.076968,Coffee Shop


In [27]:
%timeit categories = ['Coffee Shop','Café','Dessert Shop','Vegetarian / Vegan Restaurant','Fondue Restaurant','Gluten-free Restaurant']

49.2 ns ± 0.435 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [28]:
%timeit neworleans_venues[neworleans_venues['venue_category'].isin(categories)]

259 µs ± 29.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


##### Set venue_category as the index and use .loc to match index values to desired categories

In [29]:
venuecat_index = neworleans_venues
venuecat_index.index = venuecat_index['venue_category']
venuecat_index.loc[['Coffee Shop','Café','Dessert Shop','Vegetarian / Vegan Restaurant','Fondue Restaurant','Gluten-free Restaurant']]

,zipcode,zip_lat,zip_long,venue,venue_lat,venue_long,venue_category
venue_category,,,,,,,
Coffee Shop,70115,29.927305,-90.10197,Mojo Coffee House,29.935092,-90.106383,Coffee Shop
Coffee Shop,70115,29.927305,-90.10197,Bearcat Cafe,29.935653,-90.104042,Coffee Shop
Coffee Shop,70115,29.927305,-90.10197,HEY! Café,29.920800,-90.101158,Coffee Shop
Coffee Shop,70115,29.927305,-90.10197,Neutral Ground Coffee House,29.930143,-90.110026,Coffee Shop
Coffee Shop,70118,29.952305,-90.12347,Zotz Cafe,29.948235,-90.130940,Coffee Shop
...,...,...,...,...,...,...,...
Vegetarian / Vegan Restaurant,70119,29.974504,-90.08747,Sweet Soul Food,29.973162,-90.080181,Vegetarian / Vegan Restaurant
Vegetarian / Vegan Restaurant,70130,29.938005,-90.07195,Seed,29.939057,-90.073527,Vegetarian / Vegan Restaurant
Vegetarian / Vegan Restaurant,70117,29.970404,-90.03036,Sneaky Pickle,29.965785,-90.034883,Vegetarian / Vegan Restaurant


In [30]:
%timeit venuecat_index = neworleans_venues

25 ns ± 0.891 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [31]:
%timeit venuecat_index.index = venuecat_index['venue_category']

93.4 µs ± 522 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [32]:
%timeit venuecat_index.loc[['Coffee Shop','Café','Dessert Shop','Vegetarian / Vegan Restaurant','Fondue Restaurant','Gluten-free Restaurant']]

269 µs ± 40.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


##### Index with the query method

In [33]:
categories = [
    'Coffee Shop',
    'Café',
    'Dessert Shop',
    'Vegetarian / Vegan Restaurant',
    'Fondue Restaurant',
    'Gluten-free Restaurant'
]
qs = ("venue_category in @categories")
venues_filtered = neworleans_venues.query(qs)
venues_filtered

,zipcode,zip_lat,zip_long,venue,venue_lat,venue_long,venue_category
venue_category,,,,,,,
Café,70115,29.927305,-90.101970,French Truck Cafe,29.929239,-90.104610,Café
Coffee Shop,70115,29.927305,-90.101970,Mojo Coffee House,29.935092,-90.106383,Coffee Shop
Dessert Shop,70115,29.927305,-90.101970,Yogurtland,29.924917,-90.108316,Dessert Shop
Coffee Shop,70115,29.927305,-90.101970,Bearcat Cafe,29.935653,-90.104042,Coffee Shop
Café,70115,29.927305,-90.101970,Rook Cafe,29.935027,-90.104727,Café
...,...,...,...,...,...,...,...
Coffee Shop,70158,29.922905,-90.070922,Common Grounds,29.915917,-90.064533,Coffee Shop
Coffee Shop,70158,29.922905,-90.070922,PJ's Coffee & Tea,29.929576,-90.076415,Coffee Shop
Coffee Shop,70158,29.922905,-90.070922,Cherry Coffee,29.929541,-90.077029,Coffee Shop


In [34]:
%timeit categories = ['Coffee Shop','Café','Dessert Shop','Vegetarian / Vegan Restaurant','Fondue Restaurant','Gluten-free Restaurant']

52.7 ns ± 1.55 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [35]:
%timeit qs = ("venue_category in @categories")

14.2 ns ± 0.523 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [36]:
%timeit venues_filtered = neworleans_venues.query(qs)

1.06 ms ± 27.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [37]:
%timeit venues_filtered

21.8 ns ± 0.226 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Based on this analysis, it would appear that using the .isin method with boolean selection takes the least computational time.  It's also simple to set up.  This will be my preferred method going forward if the criteria is contained to filtering by row values for one column.  While setting the venue_category as index was also relatively inexpensive from a computational perspective, it doesn't allow me to use the same method for filtering more than 1 row.  If I need to filter by row values in more than one column I can either construct multiple boolean conditions or set up multiple conditions in qs using index with the query method.  Both methods were relatively close from a computational perspective.